In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [3]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [4]:
SYSTEM_PROMPT = """
I want you to act as software tester.
Your task is to read this information about a use case.
Then you predict all scenarios that can happen in this use case.

Rules of generating scenarios you should follow:
- Positive test scenarios are scenarios generating from main flow or basic fow or alternative flows. Only create one test scenarios to test the baisc flow (main flow) with all valid conditions.
- Negative test scenarios are scenarios generating from exception flows. One exception flow or one alternative flow can have multiple test scenarios according to the conditions extracted from the flow. Each invalid condition in flows must have a test scenario.
- Scenarios involve executing all steps within a specific flow with varied data or actions.
- Scenarios should be independent and require a complete flow to proceed.
- A scenario encompasses a whole function, not just verifying individual steps.
- Each flow must have at least one test scenario to cover the flow.
"""

In [5]:
usecase_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\dataset\SpecificationData\Shopping System"
save_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\prove\Shopping System\ts"

In [6]:
usecase_name_list = []
project_name = os.path.basename(usecase_path)

In [7]:
for filename in os.listdir(usecase_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

['add to cart', 'checkout', 'product information display', 'purchase', 'search']


In [8]:
for usecase_name in usecase_name_list:
    print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    prompt_all = read_file_content(usecase_directlink)
    promptSYSTEM_PROMPT = [
        { "role": "system", "content": SYSTEM_PROMPT},
        { "role": "user", "content": prompt_all}
    ]
    gpt_response = ask(promptSYSTEM_PROMPT, client, model)    
    print("gpt_response \n" +gpt_response)
    write_string_to_file(os.path.join(save_path,f"{usecase_name}.txt"), gpt_response)

add to cart
gpt_response 
### Positive Test Scenarios

1. **Add Item with No Options to Cart Successfully**
   - Preconditions: User is logged into their Shopee account and views a product detail with no options and sufficient stock.
   - Steps: User views the product detail, clicks the "Add to Cart" button, and the system confirms the item is added by updating the cart icon.
   - Expected Result: The item is added to the user's shopping cart successfully, and the cart icon reflects the updated number of items.

2. **Add Item with Options to Cart Successfully**
   - Preconditions: User is logged into their Shopee account and views a product detail with options and sufficient stock.
   - Steps: User selects an available option, adjusts the quantity (if desired), clicks the "Add to Cart" button, and the system confirms the item is added by updating the cart icon.
   - Expected Result: The selected item option is added to the user's shopping cart successfully, and the cart icon reflects t